In [ ]:
import os
import re
import numpy as np
from collections import defaultdict
from math import log


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())


In [ ]:

# Load documents
def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                docs[filename] = preprocess(file.read())
    return docs


In [ ]:
# Load queries
def load_queries(query_file_path):
    with open(query_file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]


In [ ]:
# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

In [ ]:
# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

In [ ]:
import os

# Main retrieval function
def retrieve_documents(folder_path, query_file_path):
    docs = load_documents(folder_path)
    queries = load_queries(query_file_path)

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    # Define the output file path in the same directory as the dataset folder
    output_file_path = os.path.join(folder_path, 'Shirish_result.txt')

    # Open a file to write results in the same directory as the dataset
    with open(output_file_path, 'w') as result_file:
        for query in queries:
            query_terms = preprocess(query)
            scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            # Print to console and write to file
            print(f"Query: {query}")
            result_file.write(f"Query: {query}\n")

            # Print and write only the top 3 results with ranks
            for rank, (doc_id, score) in enumerate(ranked_docs[:3], start=1):
                print(f"Rank {rank}: Document {doc_id}, Score: {score:.4f}")
                result_file.write(f"Rank {rank}: Document {doc_id}, Score: {score:.4f}\n")

            print()
            result_file.write("\n")


folder_path = '/content/drive/MyDrive/info_retrieval_lab/Final project/dataset'
query_file_path = '/content/drive/MyDrive/info_retrieval_lab/Final project/queries.txt'
retrieve_documents(folder_path, query_file_path)


Query: {\rtf1\ansi\ansicpg1252\cocoartf2709
Rank 1: Document Great_Dane.txt, Score: 0.4478
Rank 2: Document Labrador.txt, Score: 0.4127
Rank 3: Document Beagle.txt, Score: 0.3810

Query: \cocoatextscaling0\cocoaplatform0{\fonttbl\f0\froman\fcharset0 Times-Roman;}
Rank 1: Document Great_Dane.txt, Score: 0.2005
Rank 2: Document Labrador.txt, Score: 0.1703
Rank 3: Document Beagle.txt, Score: 0.1452

Query: {\colortbl;\red255\green255\blue255;\red0\green0\blue0;}
Rank 1: Document Great_Dane.txt, Score: 0.2451
Rank 2: Document Labrador.txt, Score: 0.2125
Rank 3: Document Beagle.txt, Score: 0.1848

Query: {\*\expandedcolortbl;;\cssrgb\c0\c0\c0;}
Rank 1: Document Great_Dane.txt, Score: 0.3663
Rank 2: Document Labrador.txt, Score: 0.3308
Rank 3: Document Beagle.txt, Score: 0.2993

Query: \paperw11900\paperh16840\margl1440\margr1440\vieww11520\viewh8400\viewkind0
Rank 1: Document Great_Dane.txt, Score: 0.2451
Rank 2: Document Labrador.txt, Score: 0.2125
Rank 3: Document Beagle.txt, Score: 0.184